<a href="https://colab.research.google.com/github/ELehmann91/NLP-Contract-Analysis/blob/master/extract_fields.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [0]:
%%capture#
!pip install PyPDF2
!pip install spacy
!pip install spacy-langdetect
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

In [0]:
import os
import json
import pandas as pd
import PyPDF2
import re
import spacy
nlp_en = spacy.load("en_core_web_sm")
nlp_de = spacy.load("de_core_news_sm")
from spacy.matcher import PhraseMatcher
from spacy_langdetect import LanguageDetector

## Read PDF

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
path = '/content/gdrive/Shared drives/FS-AI Base Data/innolytiq_pdf/'
file_list = os.listdir(path)
len(file_list)

149

In [52]:
file = file_list[89]

with open(path+file+'/'+file+'.pdf', 'rb') as pdfFileObj:
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    try:
        pdf_text = ' '.join([pdfReader.getPage(i).extractText() for i in range(0,min(pdfReader.numPages,50))]).replace('\n',' ')
    except:
      print('cant read '+file)

print('name:',file,'words:',len(pdf_text.split()))

name: final_terms_isin_xs1529948934.pdf words: 2879


In [0]:
def proW(w):
  w = w.lower()
  w = w.strip()
  w = re.sub('[^A-Za-zÄÖÜßäöüß]+', '',w)
  return w

In [0]:
#"Festverzinsliche", 'fixed'
#pdf_text.lower().find('ausgabepreis')

In [0]:
#pdf_text[626:648]

In [0]:
#a = [ i for i,t in enumerate(pdf_text.split()) if proW(t) == 'ausgabepreis']
#a

## Functions find extraction

In [0]:
def multi_corpus(text):
  '''
  check if german or english and use spacy language model accordingly 
  return spacy doc
  '''
  nlp = spacy.load('en')
  nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)
  doc = nlp(text)
  # document level language detection. Think of it like average language of the document!
  if doc._.language['language'] == 'en':
    doc = nlp_en(text)
  elif doc._.language['language'] == 'de':
    doc = nlp_en(text)
  else:
    print('not found', doc._.language['language'])
  return doc



In [0]:
def find_ident(doc,ident_list):
    for i, token in enumerate(doc):
        if token.text in ident_list:
          window = [tok for tok in doc[i-3:i+6]]
        if doc[i].text +' '+ token.text in ident_list:
          window = [tok.text for tok in doc[i-3:i+6]]
        #print(token.text, token.pos_, token.tag_, token.dep_, token.shape_)
    return window

In [0]:
def find_number(doc):
    for tok in doc:
        if tok.tag_ == 'CD':
          return tok.text

## test

In [0]:
doc = multi_corpus(pdf_text)

In [81]:
issue_price_ident = ["Ausgabepreis", "Issue Price", "Emissionskurs"]
window = find_ident(doc,issue_price_ident)
window

[cent, .,   , Ausgabepreis, :, 100,00, %,   , Issue]

In [82]:
no = find_number(window)
no

'100,00'